In [2]:
import PIL as pil
import matplotlib.pyplot as plt
import numpy as np
import time
h = w = c = 0

def openImage(testImage):
    return(pil.Image.open(testImage))

def imageToArray(img):
    global h, w, c
    imageRgb = img.convert("RGB")
    arrImg = np.array(img, dtype=int)
    h, w, c = arrImg.shape
    return(arrImg)

def exposure(image, num):
    arrExposure = np.array([num], dtype=np.int16)
    arrImgNew = np.clip(image + arrExposure, 0, 255).astype(np.uint8)
    return pil.Image.fromarray(arrImgNew)

def contrast(image, num):
    image = image.astype(float)
    arrImgNew = np.clip(image * num, 0, 255).astype(np.uint8)
    return pil.Image.fromarray(arrImgNew)

def flip(image, type):
    if (type == "0"):
        arrImgNew = np.flip(image, axis=0).astype(np.uint8)
    elif (type == "1"):
        arrImgNew = np.flip(image, axis=1).astype(np.uint8)
    return pil.Image.fromarray(arrImgNew)

def toGray(image):
    arrImgNew = np.dot(image[:, :,:3], [0.3, 0.59, 0.11]).astype(np.uint8)
    return pil.Image.fromarray(arrImgNew)

def sepia(image):
    arrImgNew = np.zeros_like(image, dtype=np.uint8)
    arrImgNew[:, :, 0] = np.clip(np.dot(image[:, :,:3], [0.393, 0.769, 0.189]), 0, 255)
    arrImgNew[:, :, 1] = np.clip(np.dot(image[:, :,:3], [0.349, 0.686, 0.168]), 0, 255)
    arrImgNew[:, :, 2] = np.clip(np.dot(image[:, :,:3], [0.272, 0.534, 0.131]), 0, 255)
    return pil.Image.fromarray(arrImgNew)

def gaussianKernel(sigma, size):
    x = np.linspace(-(size//2), size//2, size)
    kernel = 1 / (np.sqrt(2 * np.pi) * sigma) * np.exp(-((x ** 2) / (2 * sigma ** 2)))
    kernel /= np.sum(kernel)
    return kernel

def blur(image, sigma, size):
    image = image.astype(float) / 255.0
    kernel = gaussianKernel(sigma, size)
    blurredImage = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=0, arr=image)
    blurredImage = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=1, arr=blurredImage)

    return pil.Image.fromarray((blurredImage * 255).astype(np.uint8))

def sharpen(image, sigma, size):
    identity = np.zeros(size)
    identity[size//2] = 1
    gaussian = gaussianKernel(sigma, size)
    kernel = 2*identity - gaussian
    sharpenImage = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=0, arr=image)
    sharpenImage = np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='same'), axis=1, arr=sharpenImage)
    sharpenImage = sharpenImage.clip(0, 255).astype(np.uint8)
    return pil.Image.fromarray((sharpenImage).astype(np.uint8))

def squareCrop(image, r):
    global h
    global w
    radius = int(((r/100)*h)/2)
    cropImage = image[(h//2)-radius:(h//2)+radius, (w//2)-radius:(w//2)+radius]
    return pil.Image.fromarray((cropImage).astype(np.uint8))

def circleCrop(image):
    global h
    global w
    imgXpos, imgYpos = np.mgrid[:image.shape[0], :image.shape[1]]
    mask = (imgXpos - (h/2)-0.5)**2 + (imgYpos - (w/2)-0.5)**2 < (h/2)**2
    cropImage = image * mask[:,:,np.newaxis]
    return pil.Image.fromarray((cropImage).astype(np.uint8))

def elipCrop(image):
    global h
    global w
    f1LeftX, f1LeftY = h/7, w/7
    f2LeftX, f2LeftY = h-h/7, w-w/7
    aLeft = np.sqrt((f2LeftX+f1LeftX)**2+(f2LeftY-f1LeftY)**2)/2
    bLeft = np.sqrt(aLeft**2 - ((h//2 - f1LeftX)**2+(w//2 - f1LeftY)**2))
    imgXpos, imgYpos = np.mgrid[:image.shape[0], :image.shape[1]]
    angleRadians = np.radians(45)
    maskElipLeft = ((imgYpos-(w/2)-0.5)*np.cos(angleRadians)+(imgXpos-(h/2)-0.5)*np.sin(angleRadians))**2/aLeft**2 +\
                    ((imgYpos-(w/2)-0.5)*np.sin(angleRadians)-(imgXpos-(h/2)-0.5)*np.cos(angleRadians))**2/bLeft**2 <= 1
    maskElipRight = np.flip(maskElipLeft, axis=1).astype(np.uint8)
    mask = np.logical_or(maskElipLeft, maskElipRight)
    cropImage = image * mask[:,:,np.newaxis]
    return pil.Image.fromarray((cropImage).astype(np.uint8))

def changeName(filename, addName):
    dot = filename.rfind(".")
    if dot == -1:
        newFilename = filename + "_" + addName
    else:
        newFilename = filename[:dot] + "_" + addName + filename[dot:]
    return newFilename

def main():
    imgInput = input("Input name of picture: ")
    img = openImage(imgInput)
    arrImg = imageToArray(img)

    test = True
    while(test):
        choose = input("Choose function:\n"\
                        "0: All function \n"\
                        "1: Exposure \n" \
                        "2: Contrast \n" \
                        "3: Flip \n"\
                        "4: Change to gray/sepia \n"\
                        "5: Blur/sharpen \n"\
                        "6: Crop square \n"\
                        "7: Crop circle \n"\
                        "8: Crop elip \n")
        if(choose == "1"):
            num = input("Input exposure number: ")
            start = time.time()
            newImg = exposure(arrImg, int(num))
            end = time.time()
            newImg.save(changeName(imgInput, "exposure"))
            executionTime = end - start
            print("Exposure time : ", executionTime)
        elif(choose == "2"):
            num = input("Input contrast number (float): ")
            start = time.time()
            newImg = contrast(arrImg, float(num))
            end = time.time()
            newImg.save(changeName(imgInput, "contrast"))
            executionTime = end - start
            print("Contrast time: ", executionTime)
        elif(choose == "3"):
            num = input("Choose: 0(flip horizons), 1(flip vertical), 2(all)")
            if(num == "0"):
                start = time.time()
                newImg = flip(arrImg, num)
                end = time.time()
                newImg.save(changeName(imgInput, "horizon"))
                executionTime = end - start
                print("Flip horizon time: ", executionTime)
            elif(num == "1"):
                start = time.time()
                newImg = flip(arrImg, num)
                end = time.time()
                newImg.save(changeName(imgInput, "vertical"))
                executionTime = end - start
                print("Flip vertical time: ", executionTime)
            elif(num == "2"):
                start = time.time()
                newImg = flip(arrImg, "0")
                end = time.time()
                newImg.save(changeName(imgInput, "horizon"))
                executionTime = end - start
                print("Flip horizon time: ", executionTime)
                start = time.time()
                newImg = flip(arrImg, "1")
                end = time.time()
                newImg.save(changeName(imgInput, "vertical"))
                executionTime = end - start
                print("Flip vertical time: ", executionTime)
        elif(choose == "4"):
            num = input("Choose: 0(gray), 1(sepia)")
            if(num == "0"):
                start = time.time()
                newImg = toGray(arrImg)
                end = time.time()
                newImg.save(changeName(imgInput, "gray"))
                executionTime = end - start
                print("Change to gray time: ", executionTime)
            elif(num == "1"):
                start = time.time()
                newImg = sepia(arrImg)
                end = time.time()
                newImg.save(changeName(imgInput, "sepia"))
                executionTime = end - start
                print("Change to sepia time: ", executionTime)
        elif(choose == "5"):
            num = input("Choose: 0(blur), 1(sharpen)")
            if(num == "0"):
                sigma, size = input("Input for blur: sigma (interger), size (odd interger)(ex: 100 20): ").split()
                start = time.time()
                newImg = blur(arrImg, int(sigma), int(size))
                end = time.time()
                newImg.save(changeName(imgInput, "blur"))
                executionTime = end - start
                print("Blur time: ", executionTime)
            elif(num == "1"):
                sigma, size = input("Input for sharpen: sigma (interger), size (odd interger)(ex: 100 20): ").split()
                start = time.time()
                newImg = sharpen(arrImg, int(sigma), int(size))
                end = time.time()
                newImg.save(changeName(imgInput, "sharpen"))
                executionTime = end - start
                print("Sharpen time: ", executionTime)
        elif(choose == "6"):
            num = input("Input percent (ex: 50): ")
            start = time.time()
            newImg = squareCrop(arrImg, int(num))
            end = time.time()
            newImg.save(changeName(imgInput, "square_crop"))
            executionTime = end - start
            print("Square crop time: ", executionTime)
        elif(choose == "7"):
            start = time.time()
            newImg = circleCrop(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "circle_crop"))
            executionTime = end - start
            print("Circle crop time: ", executionTime)
        elif(choose == "8"):
            start = time.time()
            newImg = elipCrop(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "elip_crop"))
        elif(choose == "0"):
            start = time.time()
            newImg = exposure(arrImg, 50)
            end = time.time()
            newImg.save(changeName(imgInput, "exposure"))
            executionTime = end - start
            print("Exposure time: ", executionTime)

            start = time.time()
            newImg = contrast(arrImg, 1.2)
            end = time.time()
            newImg.save(changeName(imgInput, "contrast"))
            executionTime = end - start
            print("Contrast time: ", executionTime)

            start = time.time()
            newImg = flip(arrImg, "0")
            end = time.time()
            newImg.save(changeName(imgInput, "horizon"))
            executionTime = end - start
            print("Flip horizon time: ", executionTime)

            start = time.time()
            newImg = flip(arrImg, "1")
            end = time.time()
            newImg.save(changeName(imgInput, "vertical"))
            executionTime = end - start
            print("Flip vertical time: ", executionTime)

            start = time.time()
            newImg = toGray(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "gray"))
            executionTime = end - start
            print("Change to gray time: ", executionTime)

            start = time.time()
            newImg = sepia(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "sepia"))
            executionTime = end - start
            print("Change to sepia time: ", executionTime)

            start = time.time()
            newImg = blur(arrImg, 100, 21)
            end = time.time()
            newImg.save(changeName(imgInput, "blur"))
            executionTime = end - start
            print("Blur time: ", executionTime)
            
            start = time.time()
            newImg = sharpen(arrImg, 100, 21)
            end = time.time()
            newImg.save(changeName(imgInput, "sharpen"))
            executionTime = end - start
            print("Sharpen time: ", executionTime)

            start = time.time()
            newImg = squareCrop(arrImg, 50)
            end = time.time()
            newImg.save(changeName(imgInput, "square_crop"))
            executionTime = end - start
            print("Square crop time: ", executionTime)

            start = time.time()
            newImg = circleCrop(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "circle_crop"))
            executionTime = end - start
            print("Circle crop time: ", executionTime)

            start = time.time()
            newImg = elipCrop(arrImg)
            end = time.time()
            newImg.save(changeName(imgInput, "elip_crop"))
            executionTime = end - start
            print("Elip crop time: ", executionTime)
        outTest = input("Continue or exit: 0(continue), 1(exit)")
        if(outTest == "1"):
            test = False

main()

Exposure time:  0.04385018348693848
Contrast time:  0.04932594299316406
Flip horizon time:  0.00929880142211914
Flip vertical time:  0.01696181297302246
Change to gray time:  0.04704546928405762
Change to sepia time:  0.21255707740783691
Blur time:  0.33978700637817383
Sharpen time:  0.3084580898284912
Square crop time:  0.0010380744934082031
Circle crop time:  0.0721585750579834
Elip crop time:  0.13712549209594727
